In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential, Model, Input, load_model
from tensorflow.python.framework.ops import Tensor
from keras.layers import Dense, Dropout, Flatten, Activation, Average
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
import tensorboard 
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.applications import VGG16
import matplotlib
from matplotlib import pyplot as plt 
from sklearn import model_selection, metrics
from keras.utils import to_categorical
import seaborn as sn
import pickle
from statistics import mode
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import sklearn
import glob
from numpy import ravel

#If you want to save weight files for future use

In [ ]:
CONV_POOL_CNN_WEIGHT_FILE = os.path.join(os.getcwd(), 'conv_pool_cnn_pretrained_weights.hdf5')
ALL_CNN_WEIGHT_FILE = os.path.join(os.getcwd(), 'all_cnn_pretrained_weights.hdf5')
NIN_CNN_WEIGHT_FILE = os.path.join(os.getcwd(), 'nin_cnn_pretrained_weights.hdf5')
NEW_NIN_CNN_WEIGHT_FILE = os.path.join(os.getcwd(), 'new_nin_cnn_pretrained_weights.hdf5')

#Dataset filepaths

In [ ]:
img_filename = '/content/drive/My Drive/multi_bee_images_1_highres.pickle'
second_img_filename = '/content/drive/My Drive/multi_bee_images_2_highres.pickle'

label_filename = '/content/drive/My Drive/multi_labels_1.pickle'
second_label_filename = '/content/drive/My Drive/multi_labels_2.pickle'

"""img_filename = 'images_1_highres.pickle'
second_img_filename = 'images_2_highres.pickle'

label_filename = 'labels_1.pickle'
second_label_filename = 'labels_2.pickle'"""

with open(img_filename, 'rb') as source:
    first_imgs = pickle.load(source)
with open(second_img_filename, 'rb') as source:
    second_imgs = pickle.load(source)

X = np.concatenate((first_imgs,second_imgs),axis=0)

with open(label_filename, 'rb') as source:
    first_labels = pickle.load(source)
with open(second_label_filename, 'rb') as source:
    second_labels = pickle.load(source)

y = np.concatenate((first_labels,second_labels),axis=0)
print(set(y))

{'1 Mixed local stock 2', 'Bombus bimaculatus', 'Bombus perplexus', 'Russian honey bee', 'Bombus affinis', 'Western honey bee', 'Bombus griseocollis', 'Bombus citrinus', 'Bombus auricomus', 'Italian honey bee', 'Carniolan honey bee', 'Apis mellifera', 'Bombus impatiens', 'Bombus vagans', 'Bombus rufocinctus', 'VSH Italian honey bee', 'Bombus fraternus', 'Bombus fervidus', 'Bombus pensylvanicus'}


#Remove classes with under 200 samples


In [ ]:
classes_to_remove = ['Bombus fraternus','Bombus perplexus']
indices_to_remove = []
for i,label in enumerate(y):
  if label in classes_to_remove:
    indices_to_remove.append(i)

y = np.delete(y,indices_to_remove)
X = np.delete(X,indices_to_remove,axis=0)

#rename apis mellifera to western honey bee
y = np.where(y=='Apis mellifera','Western honey bee',y)
print(set(y))

{'1 Mixed local stock 2', 'Bombus bimaculatus', 'Russian honey bee', 'Bombus affinis', 'Western honey bee', 'Bombus griseocollis', 'Bombus citrinus', 'Bombus auricomus', 'Italian honey bee', 'Carniolan honey bee', 'Bombus impatiens', 'Bombus vagans', 'Bombus rufocinctus', 'VSH Italian honey bee', 'Bombus fervidus', 'Bombus pensylvanicus'}


In [ ]:
bee_names = np.array(y)
assert(len(X)==len(y))
number_classes = len(set(y))

#Undersample classes over 1,000 down to 1,000

In [ ]:
undersampling_counts = {'1 Mixed local stock 2': 472,
 'Bombus affinis': 462,
 'Bombus auricomus': 1000,
 'Bombus bimaculatus': 1000,
 'Bombus citrinus': 241,
 'Bombus fervidus': 774,
 'Bombus griseocollis': 1000,
 'Bombus impatiens': 1000,
 'Bombus pensylvanicus': 742,
 'Bombus rufocinctus': 199,
 'Bombus vagans': 287,
 'Carniolan honey bee': 501,
 'Italian honey bee': 1000,
 'Russian honey bee': 527,
 'VSH Italian honey bee': 199,
 'Western honey bee': 1000}

In [ ]:
#Undersampling 
index_list = np.array([i for i in range(len(X))])
rus = RandomUnderSampler(random_state=42, sampling_strategy=undersampling_counts)
 
Xresampled, yresampled = rus.fit_resample(
    index_list.reshape(-1, 1), y
)
Xresampled = Xresampled.flatten()
print(Xresampled.shape)
Xresampled = X[Xresampled]

(10404,)


#SMOTE oversampling up to 1000 samples



In [ ]:
index_list_again = np.array([i for i in range(len(Xresampled))])
sm = SMOTE(random_state=42)
 
Xresampled_again, yresampled_again = sm.fit_resample(index_list_again.reshape(-1, 1), yresampled)
Xresampled_again = Xresampled_again.flatten()
print(Xresampled_again.shape)
Xresampled_again = Xresampled[Xresampled_again]

(16000,)


#Make text labels into numbers for one-hot encoding

In [ ]:
label_to_number = {'1 Mixed local stock 2': 0,
 'Bombus affinis': 1,
 'Bombus auricomus': 2,
 'Bombus bimaculatus': 3,
 'Bombus citrinus': 4,
 'Bombus fervidus': 5,
 'Bombus griseocollis': 6,
 'Bombus impatiens': 7,
 'Bombus pensylvanicus': 8,
 'Bombus rufocinctus': 9,
 'Bombus vagans': 10,
 'Carniolan honey bee': 11,
 'Italian honey bee': 12,
 'Russian honey bee': 13,
 'VSH Italian honey bee': 14,
 'Western honey bee': 15}

inv_bee_dict = {v:k for k,v in label_to_number.items()}

In [ ]:
number_y = []
for label in yresampled_again:
  new_label = label_to_number[label]
  number_y.append(new_label)

#Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    Xresampled_again, number_y, test_size=0.2, random_state=42
)

print(f'Files in train: {len(X_train)}. Files in test: {len(X_test)}')

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
np.unique(y_test)
input_shape = X_train[0,:,:,:].shape
model_input = Input(shape=input_shape)
print(f'Input shape: {input_shape}')
print(f'Model input: {model_input}')

Files in train: 12800. Files in test: 3200
Input shape: (64, 64, 3)
Model input: Tensor("input_2:0", shape=(?, 64, 64, 3), dtype=float32)


#Image augmentation

In [ ]:
datagen = image.ImageDataGenerator(
    rotation_range=40,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

datagen.fit(X_train)

#If you want to save the model, not the weights. Choose between data augmentation and no data augmention. Returns 2 objects: model and history

In [ ]:
def NO_WEIGHTS_compile_and_train(model,model_name, num_epochs): 
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc']) 
    #history = model.fit_generator(datagen.flow(X_train,y_train,batch_size=32),steps_per_epoch=(len(X_train)),
     #                             validation_data=(X_test,y_test),epochs=num_epochs, verbose=1)
    history = model.fit(x=X_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1,validation_split=0.15)
    model.save(model_name +'.h5')
    return model,history

#This will save your weight files. Choose between data augmentation and no data augmention. Returns 3 objects: model, history and weight file

In [ ]:
def compile_and_train(model,model_name,num_epochs): 
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc']) 
    filepath = model_name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
    tensor_board = keras.callbacks.TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    #history = model.fit_generator(datagen.flow(X_train,y_train,batch_size=32),steps_per_epoch=(len(X_train)),
     #                             validation_data=(X_test,y_test),epochs=num_epochs, verbose=1,callbacks=[checkpoint, tensor_board])
    history = model.fit(x=X_train, y=y_train, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board])
    weight_files = glob.glob(os.path.join(os.getcwd(), 'weights/*'))
    weight_file = max(weight_files, key=os.path.getctime) # most recent file
    return model,history, weight_file

#Visualizations

In [ ]:
def make_training_loss_accuracy(model_history, epochs):
  ax = plt.subplot(111)
  # Hide the right and top spines
  ax.spines['right'].set_visible(False)
  ax.spines['top'].set_visible(False)
  ax.plot(list(range(1,epochs + 1)), model_history.history['acc'], label = 'training accuracy')
  ax.plot(list(range(1,epochs + 1)), model_history.history['loss'], label = 'training loss')
  plt.xlabel('epoch number', fontsize = 16)
  plt.ylabel('training accuracy', fontsize = 14)
  ax.set_title("Training Accuracy and Loss per Epoch", fontsize = 20, pad = 40)
  plt.xticks(list(range(5, epochs+1, 5)))
  plt.legend()
  plt.show()

In [ ]:
def make_ypred_and_ytest(model,X_test,y_test):
    model.evaluate(X_test,y_test)
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred,axis=1)
    #reduce the one hot encodings
    y_truth = np.where(y_test==1)[1]
    return y_pred, y_test

In [ ]:
def make_classifiation_report_and_cm(y_pred,y_test,inv_bee_dict):
  y_test = np.where(y_test==1)[1]
  classes = set(y_test)
  print(classification_report(y_test, y_pred))
  bee_list = []
  for i in range(len(classes)):
      bee_list.append(inv_bee_dict[i])
      
  cm = metrics.confusion_matrix(y_test, y_pred)
  sn_cm = sn.heatmap(cm, annot=False,xticklabels=bee_list,yticklabels=bee_list)
  print(sn_cm)

In [ ]:
# Compute ROC curve and ROC area for each class

def make_ROC(y_pred,y_test,inv_bee_dict):
  y_truth = np.where(y_test==1)[1]
  classes = set(y_truth)
  lb = sklearn.preprocessing.LabelBinarizer()
  y_truth = lb.fit_transform(y_test)
  y_pred = lb.fit_transform(y_pred)
  fpr = dict()
  tpr = dict()
  roc_auc = dict()

  for i in range(len(classes)):
      fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(y_truth[:, i], y_pred[:,i])
      roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = sklearn.metrics.roc_curve(y_truth.ravel(), y_pred.ravel())
  roc_auc["micro"] = sklearn.metrics.auc(fpr["micro"], tpr["micro"])
  plt.figure(figsize=(12,12))
  lw = 1
  for i in range(len(classes)):
    class_name = inv_bee_dict[i]
    plt.plot(fpr[i], tpr[i], 
            color=np.random.rand(3,),lw=lw, label=class_name + ' ROC AUC = %0.2f' % roc_auc[i])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate/1-Specificity')
    plt.ylabel('True Positive Rate/Sensitivity')
    plt.title('ROC - mini batches adam optimizer with dropout, and batch normalization')
    plt.legend(loc="lower right")
  plt.show()

#Convoluational Pool CNN

In [ ]:
def conv_pool_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(16, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='conv_pool_cnn')
    
    return model

In [ ]:
NUM_EPOCHS = 20
conv_pool_cnn_model = conv_pool_cnn(model_input)
conv_model, conv_history, conv_pool_cnn_weight_file = compile_and_train(conv_pool_cnn_model, 'conv_pool_cnn', NUM_EPOCHS)

Epoch 1/20
12800/12800 [==============================] - 21s 2ms/step - loss: 2.3006 - acc: 0.1641


OSError: ignored

In [ ]:
make_training_loss_accuracy(conv_history,20)
conv_y_pred,conv_y_truth = make_ypred_and_ytest(conv_model,X_test,y_test)
make_classifiation_report_and_cm(conv_y_pred,conv_y_truth,inv_bee_dict)
make_ROC(conv_y_pred,conv_y_truth,inv_bee_dict)

#All CNN model

In [ ]:
def all_cnn(model_input):
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(16, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='all_cnn')
    
    return model


In [ ]:
NUM_EPOCHS = 20
all_cnn_model = all_cnn(model_input)
cnn_model, cnn_history, all_cnn_weight_file = compile_and_train(all_cnn_model,'all_cnn', NUM_EPOCHS)

In [ ]:
make_training_loss_accuracy(cnn_history,20)
cnn_y_pred,cnn_y_truth = make_ypred_and_ytest(cnn_model,X_test,y_test)
make_classifiation_report_and_cm(cnn_y_pred,cnn_y_truth,inv_bee_dict)
make_ROC(cnn_y_pred,cnn_y_truth,inv_bee_dict)

#NIN Model

In [ ]:
def nin_cnn(model_input):
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(16, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model


In [ ]:
NUM_EPOCHS = 20
nin_cnn_model = nin_cnn(model_input)
nin_model, nin_history, nin_cnn_weight_file = compile_and_train(nin_cnn_model,'nin',NUM_EPOCHS)

In [ ]:
make_training_loss_accuracy(nin_history,20)
nin_y_pred,nin_y_truth = make_ypred_and_ytest(nin_model,X_test,y_test)
make_classifiation_report_and_cm(nin_y_pred,nin_y_truth,inv_bee_dict)
make_ROC(nin_y_pred,nin_y_truth,inv_bee_dict)

#Improved NIN Model

In [ ]:
def new_nin_cnn(model_input):
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = Conv2D(128, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.2)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(16, (1, 1), activation='relu')(x)

    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model

In [ ]:
NUM_EPOCHS = 20
new_nin_cnn_model = new_nin_cnn(model_input)
new_nin_model, new_nin_history, new_nin_cnn_weight_file = compile_and_train(new_nin_cnn_model, 'new_nin', NUM_EPOCHS)

In [ ]:
make_training_loss_accuracy(new_nin_history,20)
new_nin_y_pred,new_nin_y_truth = make_ypred_and_ytest(new_nin_model,X_test,y_test)
make_classifiation_report_and_cm(new_nin_y_pred,new_nin_y_truth,inv_bee_dict)
make_ROC(new_nin_y_pred,new_nin_y_truth,inv_bee_dict)

#Ensemble predit

In [ ]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)
new_nin_cnn_model = new_nin_cnn(model_input)

try:
    conv_pool_cnn_model.load_weights(conv_pool_cnn_weight_file)
except NameError:
    conv_pool_cnn_model.load_weights(CONV_POOL_CNN_WEIGHT_FILE)
try:
    all_cnn_model.load_weights(all_cnn_weight_file)
except NameError:
    all_cnn_model.load_weights(ALL_CNN_WEIGHT_FILE)
try:
    nin_cnn_model.load_weights(nin_cnn_weight_file)
except NameError:
    nin_cnn_model.load_weights(NIN_CNN_WEIGHT_FILE)
try:
    new_nin_cnn_model.load_weights(new_nin_cnn_weight_file)
except NameError:
    new_nin_cnn_model.load_weights(NEW_NIN_CNN_WEIGHT_FILE)


models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model,new_nin_cnn_model]

In [ ]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [ ]:
ensemble_model = ensemble(models, model_input)

In [ ]:
def evaluate_error(model):
    pred = model.predict(X_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]
    cr = classification_report(pred,y_test)    
    return error, cr

In [ ]:
evaluate_error(ensemble_model)


17.99974817426341